In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
train_label = pd.read_csv('/kaggle/input/glaucoma-detection/glaucoma.csv')
y_train = train_label['Glaucoma']
train_label.head()

,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,OD,A,0
1,002.jpg,0.6953,OS,A,0
2,003.jpg,0.9629,OS,A,0
3,004.jpg,0.7246,OD,A,0
4,005.jpg,0.6138,OS,A,0


In [3]:
positive_samples = train_label[train_label['Glaucoma'] == 1]
negative_samples = train_label[train_label['Glaucoma'] == 0]

# Print the number of images for positive and negative classes
print(f"Number of positive samples: {len(positive_samples)}")
print(f"Number of negative samples: {len(negative_samples)}")

Number of positive samples: 168
Number of negative samples: 482


In [4]:
from numpy import asarray
from PIL import Image

# Open random image for checking details
image = Image.open('/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Positive/613.jpg')

# summarize some details about the image
print("Format: ", image.format)
print("Mode: ", image.mode)
print("Size:", image.size)
# show the image
image.show()
pixels = asarray(image)

Format:  JPEG
Mode:  RGB
Size: (3072, 2048)


# **Augmentation**

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

In [6]:
TRAIN_DIR = '/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Train'

TEST_DIR = '/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation'

# **Utilizing ResNet50**

In [7]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model 
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard
import keras
import matplotlib.pyplot as plt

Image size taken as 224x224 square

In [8]:
HEIGHT = 224
WIDTH = 224

Important Parameters and tuning

In [9]:
BATCH_SIZE = 32
class_list = ["class_1", "class_2"]
FC_LAYERS = [1024, 512, 256]
dropout = 0.5
NUM_EPOCHS = 30
STEPS_PER_EPOCH = 150

In [10]:
def build_model(base_model, dropout, fc_layers, num_classes):
    #freeze
    for layer in base_model.layers:
        layer.trainable = False
    
    #basemodel to flatten to 1d 
    x = base_model.output
    x = Flatten()(x)
    
    # add fully connected layer
    for fc in fc_layers:
        print(fc)
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)
        
    # last fc layer for prediction
    preditions = Dense(num_classes, activation='softmax')(x)
    
    finetune_model = Model(inputs = base_model.input, outputs = preditions)
    
    return finetune_model

In [11]:
base_model_1 = ResNet50(weights = 'imagenet',
                       include_top = False,
                       input_shape = (HEIGHT, WIDTH, 3))

94773248/94765736 [==============================] - 1s 0us/step


In [12]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                   rotation_range = 90,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.1,)

test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                  rotation_range = 90,
                                  horizontal_flip = True,
                                  vertical_flip = False)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size = (HEIGHT, WIDTH),
                                                    batch_size = BATCH_SIZE)

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  target_size = (HEIGHT, WIDTH),
                                                  batch_size = BATCH_SIZE)

Found 520 images belonging to 2 classes.
Found 130 images belonging to 2 classes.


In [13]:
resnet50_model = build_model(base_model_1, dropout = dropout,
                                      fc_layers = FC_LAYERS,
                                      num_classes = len(class_list))

1024
512
256


In [14]:
adam = Adam(lr = 0.00001)
resnet50_model.compile(adam, loss="binary_crossentropy", metrics=["accuracy"])

filepath = "./checkpoints" + "RestNet50" + "_model_weights.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor = ["acc"], verbose= 1, mode = "max")
cb=TensorBoard(log_dir=("/home/ubuntu/"))
callbacks_list = [checkpoint, cb]

print(train_generator.class_indices)

resnet50_model.summary()

{'Glaucoma_Negative': 0, 'Glaucoma_Positive': 1}
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________

In [15]:
history = resnet50_model.fit_generator(generator = train_generator, epochs = NUM_EPOCHS, steps_per_epoch = STEPS_PER_EPOCH, 
                                       shuffle = True, validation_data = test_generator)

Epoch 1/30
 17/150 [==>...........................] - 56s 3s/step - loss: 1.6215 - accuracy: 0.6096 - val_loss: 1.0856 - val_accuracy: 0.7385


In [16]:
# Extract and print accuracy and loss
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

print(f"Training Accuracy: {training_accuracy[-1]:.4f}")
print(f"Validation Accuracy: {validation_accuracy[-1]:.4f}")
print(f"Training Loss: {training_loss[-1]:.4f}")
print(f"Validation Loss: {validation_loss[-1]:.4f}")

accuracy = (validation_accuracy[-1] * 100)
print(f"\n\nModel Accuracy: {accuracy:.2f}%")

Training Accuracy: 0.6096
Validation Accuracy: 0.7385
Training Loss: 1.6215
Validation Loss: 1.0856


Model Accuracy: 73.85%
